In [6]:
pip install autocorrect

In [8]:
import pandas as pd

# Load the amazon review dataset

df = pd.read_csv('https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/amazon.csv')#[0:2000]

df

,reviewText,Positive
0,This is a one of the best apps acording to a b...,1
1,This is a pretty good version of the game for ...,1
2,this is a really cool game. there are a bunch ...,1
3,"This is a silly game and can be frustrating, b...",1
4,This is a terrific game on any pad. Hrs of fun...,1
...,...,...
19995,this app is fricken stupid.it froze on the kin...,0
19996,Please add me!!!!! I need neighbors! Ginger101...,1
19997,love it! this game. is awesome. wish it had m...,1
19998,I love love love this app on my side of fashio...,1


In [10]:
import re
from autocorrect import Speller
spell = Speller(lang='en')
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemm = WordNetLemmatizer()

#Fixing Word Lengthening
def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

def text_preprocess(doc):
    #Lowercasing all the letters
    temp = doc.lower()
    #Removing hashtags and mentions
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    #Removing links
    temp = re.sub(r"http\S+", "", temp)
    temp = re.sub(r"www.\S+", "", temp)
    #removing numbers
    temp = re.sub("[0-9]","", temp)
    #Removing '
    temp = re.sub("'"," ",temp)

    #Tokenization
    #temp = word_tokenize(temp)
    #Fixing Word Lengthening
    temp = [reduce_lengthening(w) for w in temp]
    #spell corrector
    temp = [spell(w) for w in temp]
    #stem
    temp = [lemm.lemmatize(w) for w in temp]
    #Removing short words
    temp = [w for w in temp if len(w)>2]
    temp = " ".join(w for w in temp)

    return temp
df['reviewText'] = df['reviewText'].apply(text_preprocess)

from sklearn.model_selection import train_test_split
#replacing textual categories by integers

df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,reviewText,Positive
0,,1
1,,1
2,,1
3,,1
4,,1
...,...,...
19995,,0
19996,,1
19997,,1
19998,,1


In [11]:
#allora io sto usando bert per la tokenizzazione. E' un modello subottimale. esiste quello gpt che è il seguente
'''from collections import Counter, defaultdict
import re

# Funzione per ottenere le statistiche delle coppie di caratteri
def get_stats(corpus):
    pairs = defaultdict(int)
    for word in corpus:
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[(symbols[i], symbols[i + 1])] += 1
    return pairs

# Funzione per eseguire l'algoritmo BPE
def bpe(corpus, num_operations):
    vocab = Counter(corpus)
    for _ in range(num_operations):
        pairs = get_stats(corpus)
        if not pairs:
            break
        best_pair = max(pairs, key=pairs.get)
        new_vocab = Counter()
        bigram = re.compile(r'(?<!\S)' + re.escape(' '.join(best_pair)) + r'(?!\S)')
        for word in vocab:
            new_word = bigram.sub(''.join(best_pair), word)
            new_vocab[new_word] += vocab[word]
        vocab = new_vocab
    return vocab

# Funzione per tokenizzare il testo utilizzando il vocabolario BPE
def tokenize_bpe(text, vocab):
    tokens = text.split()
    new_tokens = []
    for token in tokens:
        while True:
            pairs = get_stats([token])
            if not pairs:
                break
            best_pair = max(pairs, key=pairs.get)
            if best_pair in vocab:
                token = token.replace(''.join(best_pair), ' '.join(best_pair))
            else:
                break
        new_tokens.extend(token.split())
    return new_tokens
# Supponiamo che tu abbia già un corpus di testo
corpus = ["word1 word2", "word3 word4"]  # Sostituisci con il tuo corpus
bpe_vocab = bpe(corpus, num_operations=1000)
df = pd.DataFrame({
    'text': ["sample text 1", "sample text 2"],
    'label': [0, 1]
})

# Inizializzazione del vocabolario BPE (già addestrato)
# bpe_vocab è un dizionario di token BPE

# Tokenizzazione BPE
def tokenize(texts, vocab, max_length=128):
    tokenized_texts = [tokenize_bpe(text, vocab) for text in texts]
    # Padding per uniformare la lunghezza
    padded_texts = [tokens + ['<pad>'] * (max_length - len(tokens)) for tokens in tokenized_texts]
    return torch.tensor(padded_texts)'''
#GPT:
'''Testo: "lower"
Iniziale: ['l', 'o', 'w', 'e', 'r']
Operazioni BPE: Supponiamo che dopo alcune iterazioni, le coppie "lo" e "we" siano le più frequenti.
Token Finale: ['l', 'o', 'w', 'er'] (dove "lo" e "we" sono stati fusi in token unici)
#BERT
Testo: "lower"
Iniziale: ['low', '##er'] (dove "##" indica che "er" è un sottotoken di "low")
Token Finale: ['low', '##er']'''

'Testo: "lower"\nIniziale: [\'l\', \'o\', \'w\', \'e\', \'r\']\nOperazioni BPE: Supponiamo che dopo alcune iterazioni, le coppie "lo" e "we" siano le più frequenti.\nToken Finale: [\'l\', \'o\', \'w\', \'er\'] (dove "lo" e "we" sono stati fusi in token unici)\n#BERT\nTesto: "lower"\nIniziale: [\'low\', \'##er\'] (dove "##" indica che "er" è un sottotoken di "low")\nToken Finale: [\'low\', \'##er\']'

In [12]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split

# Inizializzazione del tokenizer
'''tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenizzazione e padding
def tokenize(texts, max_length=128):#significa che torno il testo dopo 128 caratteri. Convine solo per avere meno complessità e piu' memoria
    encoding = tokenizer(texts, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')
    return encoding['input_ids'], encoding['attention_mask']


# Tokenizzazione
X = df.iloc[:,0]
y = df.iloc[:,1]
X = X.tolist()  # Se X è un array NumPy

input_ids, attention_mask = tokenize(X)
labels = torch.tensor(y)

# Suddivisione del dataset
X_train, X_test, y_train, y_test = train_test_split(input_ids, labels, test_size=0.2, random_state=42, shuffle=False)
attention_mask_train, attention_mask_test = train_test_split(attention_mask, test_size=0.2, random_state=42, shuffle=False)

# Creazione dei TensorDataset
train_dataset = TensorDataset(X_train, attention_mask_train, y_train)
test_dataset = TensorDataset(X_test, attention_mask_test, y_test)

# Creazione dei DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)'''

import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
import pandas as pd

# Inizializzazione del tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenizzazione e padding
def tokenize(texts, max_length=128):
    """
    Tokenizza e applica padding ai testi, troncandoli a max_length.
    """
    encoding = tokenizer(texts, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')
    return encoding['input_ids'], encoding['attention_mask']

# Funzione per creare finestre mobili temporali
def create_sliding_windows(X, y, window_size):
    input_ids_list = []
    attention_mask_list = []
    labels_list = []

    for i in range(len(X) - window_size + 1):
        window = X[i:i + window_size]
        input_ids, attention_mask = tokenize(window)

        # Aggiungi la finestra corrente ai dati
        input_ids_list.append(input_ids)
        attention_mask_list.append(attention_mask)
        labels_list.append(y[i + window_size - 1])

    return input_ids_list, attention_mask_list, labels_list

# Supponiamo che il DataFrame abbia due colonne: "sentence" e "label"
X = df.iloc[:,0]
y = df.iloc[:,1]
X = X.tolist()
input_ids_list, attention_mask_list, labels_list = create_sliding_windows(X, y, window_size=2)

# Suddivisione del dataset in train e test (assicurati che il numero di campioni sia consistente)
X_train, X_test, y_train, y_test = train_test_split(input_ids_list, labels_list, test_size=0.2, random_state=42, shuffle=False)
attention_mask_train, attention_mask_test = train_test_split(attention_mask_list, test_size=0.2, random_state=42, shuffle=False)

# Creazione dei TensorDataset (converti le liste di finestre in tensori per DataLoader)
train_dataset = TensorDataset(torch.stack(X_train), torch.stack(attention_mask_train), torch.tensor(y_train))
test_dataset = TensorDataset(torch.stack(X_test), torch.stack(attention_mask_test), torch.tensor(y_test))

# Creazione dei DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [13]:
#IN PRATICA LA DIMENISONALITÀ DEL SISTEMA È DATA SOLO DAL NUMERO DI TOKEN. MOLTO PROABILMETE LA RELATÀ ABITA IN DIMENSIONI INFERIORI.

In [14]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(512.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(1)]

class NewsEmbedding(nn.Module):
    def __init__(self, vocab_size, d_model):
        super(NewsEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model)

    def forward(self, x):
        #print(self.embedding(x).shape)
        #print(self.positional_encoding(x).shape)
        return self.embedding(x) + self.positional_encoding(x).unsqueeze(-2)

class NewsAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super(NewsAttention, self).__init__()
        self.multihead_attn = nn.MultiheadAttention(d_model, n_heads)

    def forward(self, x):
        return self.multihead_attn(x, x, x)[0]

class NewsTransformerLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff=2048, dropout=0.1):
        super(NewsTransformerLayer, self).__init__()
        self.self_attn = NewsAttention(d_model, n_heads)
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model)
        )
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.norm1(x + self.dropout(self.self_attn(x)))
        x = self.norm2(x + self.dropout(self.feed_forward(x)))
        return x

class NewsModel(nn.Module):
    def __init__(self, vocab_size, d_model=32, n_heads=1, n_layers=1, dropout=0.01):
        super(NewsModel, self).__init__()
        self.embedding = NewsEmbedding(vocab_size, d_model)
        self.layers = nn.ModuleList([NewsTransformerLayer(d_model, n_heads, dropout=dropout) for _ in range(n_layers)])
        self.final_layer = nn.Linear(d_model, 1)  # Output layer for binary classification
        self.final_layer2 = nn.Linear(256, 1)  # Output layer for binary classification
        self.sigmoid = nn.Sigmoid()  # Sigmoid activation

    def forward(self, x):

        x = self.embedding(x)
        x=x.reshape(x.shape[0],x.shape[1]*x.shape[2],x.shape[3])
        #print(x.shape)
        for layer in self.layers:
            #print(x.shape)

            x = layer(x)
        x = self.final_layer(x).squeeze(-1) # Pass through final layer
        #print(x.shape)
        x = self.final_layer2(x)
        #print(x.shape)
        x = self.sigmoid(x)# Apply sigmoid activation

        return x# Return the probabilities for binary classification
import torch

# Verifica se CUDA è disponibile e seleziona il dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vocab_size = tokenizer.vocab_size # Cambia secondo le tue necessità
print(device)
model = NewsModel(vocab_size).to(device)

cuda


In [17]:
# Esegui l'addestramento e la valutazione
import torch
import torch.nn as nn
import torch.cuda as cuda
from sklearn.metrics import accuracy_score
from torch.optim import Adam
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=1e-3)


def train(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids, attention_mask, labels = [tensor.to(device) for tensor in batch]
        #print(input_ids.shape)
        optimizer.zero_grad()

        outputs = model(input_ids)  # Assicurati che il modello utilizzi anche attention_mask`

        loss = criterion(outputs.squeeze(-1), labels.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

def evaluate(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [tensor.to(device) for tensor in batch]
            outputs = model(input_ids)  # Assicurati che il modello utilizzi anche attention_mask
            preds = (outputs > 0.5).float()  # Convert to binary prediction
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return accuracy_score(all_labels, all_preds)

num_epochs = 30  # Scegli il numero di epoche che preferisci

for epoch in range(num_epochs):
    train_loss = train(model, train_dataloader, optimizer, criterion)
    accuracy = evaluate(model, test_dataloader)

    print(f'Epoch {epoch + 1}/{num_epochs}')
    print(f'Train Loss: {train_loss}')
    print(f'Test Accuracy: {accuracy}')

Epoch 1/30
Train Loss: 0.5352195669412613
Test Accuracy: 0.6785
Epoch 2/30
Train Loss: 0.5303075504302979
Test Accuracy: 0.6785
Epoch 3/30
Train Loss: 0.5304490419626235
Test Accuracy: 0.6785
Epoch 4/30
Train Loss: 0.5353032834529877
Test Accuracy: 0.6785
Epoch 5/30
Train Loss: 0.5330172905921936
Test Accuracy: 0.6785
Epoch 6/30
Train Loss: 0.5316221070289612
Test Accuracy: 0.6785
Epoch 7/30
Train Loss: 0.531461767911911
Test Accuracy: 0.6785
Epoch 8/30
Train Loss: 0.5318585853576661
Test Accuracy: 0.6785
Epoch 9/30
Train Loss: 0.5326446719169616
Test Accuracy: 0.6785
Epoch 10/30
Train Loss: 0.5324921445846558
Test Accuracy: 0.6785
Epoch 11/30
Train Loss: 0.5318813147544861
Test Accuracy: 0.6785
Epoch 12/30
Train Loss: 0.532824102640152
Test Accuracy: 0.6785
Epoch 13/30
Train Loss: 0.5307587554454803
Test Accuracy: 0.6785
Epoch 14/30
Train Loss: 0.5300297753810883
Test Accuracy: 0.6785
Epoch 15/30
Train Loss: 0.5282833926677704
Test Accuracy: 0.6785
Epoch 16/30
Train Loss: 0.5290040342